# データセットと学習済み重みのダウンロード

In [1]:
'''
1. データセット用フォルダー「data」と
   学習済みの重み用フォルダー「weight」の作成
'''
import os

# 「data」フォルダーを作成
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# 「weights」フォルダーを作成
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

In [2]:
%%time
'''
2. VOC2012のデータセットを
   Visual Object Classes Challenge 2012のサイトからダウンロード
'''
import urllib.request
import tarfile

# VOCデータセットのダウンロード先のURL
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
# data_dirにファイル名を連結してファイルパスを作成
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
    # urlにアクセスしてファイルパスtarget_pathとしてダウンロード
    urllib.request.urlretrieve(url, target_path)
    # ダウンロードしたtar形式ファイルを読み込む
    tar = tarfile.TarFile(target_path)
    tar.extractall(data_dir)  # tarファイルを解凍
    tar.close()               # tarファイルをクローズ

CPU times: user 7.72 s, sys: 8.3 s, total: 16 s
Wall time: 1min 49s


In [3]:
%%time
'''
3. SSDで使用するVGG16の学習済みの重みを
  「weights」フォルダーにダウンロード
'''
# VGG16の学習済み重みのダウンロード先
url = "https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth"
# weights_dirにファイル名を連結してファイルパスを作成
target_path = os.path.join(weights_dir, "vgg16_reducedfc.pth") 

if not os.path.exists(target_path):
    # urlにアクセスしてファイルパスtarget_pathとしてダウンロード
    urllib.request.urlretrieve(url, target_path)

CPU times: user 160 ms, sys: 115 ms, total: 275 ms
Wall time: 2.92 s


In [4]:
%%time
'''
4. SSD300の学習済みの重みを
  「weights」フォルダーにダウンロード
'''
# SSD300の学習済み重みのダウンロード先
url = "https://s3.amazonaws.com/amdegroot-models/ssd300_mAP_77.43_v2.pth"
# weights_dirにファイル名を連結してファイルパスを作成
target_path = os.path.join(weights_dir, "ssd300_mAP_77.43_v2.pth") 

if not os.path.exists(target_path):
    # urlにアクセスしてファイルパスtarget_pathとしてダウンロード
    urllib.request.urlretrieve(url, target_path)

CPU times: user 217 ms, sys: 147 ms, total: 364 ms
Wall time: 3.36 s


In [7]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [15]:
! ls

data  drive  sample_data  weights


In [6]:
'''
1. GPUが使用可能であればデバイスに割り当てる
'''
import torch
# GPUが使用可能であれば割り当てる
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('使用されるデバイス：', device)

使用されるデバイス： cuda


In [9]:
'''
2. 「ObjectDetection」内のモジュールをインポート可能にする
'''
import sys

# モジュールのインポートが行えるように「VOC2021_SSDのパスを登録
# 以下のパスは環境に合わせて書き換える必要があります
sys.path.append('/content/drive/My Drive/Colab Notebooks/VOC2021_SSD')

In [10]:
'''
3. データローダーの作成
'''
import torch.utils.data as data
# vocモジュールからインポート
from voc import make_filepath_list, GetBBoxAndLabel, DataTransform, \
                PreprocessVOC2012, multiobject_collate_fn

# データセットの親フォルダーのパス
# 以下のパスは環境に合わせて書き換える必要があります
rootpath = "./data/VOCdevkit/VOC2012/"
# 訓練用、検証用の画像、アノテーションデータのファイルパスのリストを作成
train_img_list, train_anno_list, val_img_list, val_anno_list = make_filepath_list(
    rootpath)

# VOC2012の正解ラベルのリスト
voc_classes = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train',
    'tvmonitor']

# データセットのRGBの平均値(R)123,(G)117,(B)104をOpenCVのBGRの並びに合わせて
# (104, 117, 123)のタプルにする
color_mean = (104, 117, 123)

# 画像を入力サイズを300×300にする
input_size = 300

# DataTransformでVOC2012データセットを前処理して以下のデータを取得
#  ・前処理後のイメージ[R,G,B](Tensor)
#  ・BBoxとラベル(ndarray)
#  ・イメージの高さ、幅(int)
train_dataset = PreprocessVOC2012(
    train_img_list,  # イメージのパスリスト
    train_anno_list, # アノテーションのパスリスト
    phase="train",   # 訓練モード
    transform=DataTransform(input_size, color_mean), # 前処理オブジェクト
    get_bbox_label=GetBBoxAndLabel(voc_classes)) # BBoxとラベル取得

val_dataset = PreprocessVOC2012(
    val_img_list,  # イメージのパスリスト
    val_anno_list, # アノテーションのパスリスト
    phase="val",   # 訓練モード
    transform=DataTransform(input_size, color_mean), # 前処理オブジェクト
    get_bbox_label=GetBBoxAndLabel(voc_classes)) # BBoxとラベル取得

batch_size = 32 # 1バッチあたりのデータ数は32

# 訓練用のミニバッチを生成するデータローダー
train_dataloader = data.DataLoader(
    train_dataset,         # 前処理した訓練データ
    batch_size=batch_size, # バッチサイズ
    shuffle=True, # ミニバッチ抽出の際にシャッフルする
    collate_fn=multiobject_collate_fn) # ミニバッチ生成関数

# 検証用のミニバッチを生成するデータローダー
val_dataloader = data.DataLoader(
    val_dataset,           # 前処理した検証データ
    batch_size=batch_size, # バッチサイズ
    shuffle=False, # ミニバッチ抽出の際にシャッフルしない
    collate_fn=multiobject_collate_fn) # ミニバッチ生成関数

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [11]:
'''
4. torchinfoをpipコマンドでインストールする
'''
! pip install torchinfo

In [12]:
'''
5. SSDモデルの作成
'''
import torch.nn as nn
import torch.nn.init as init
from ssd import SSD # ssdモジュールのインポート
from torchinfo import summary

# SSDモデルの設定値
ssd_cfg = {
    'classes_num': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'dbox_num': [4, 6, 6, 6, 4, 4],  # DBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDモデルを生成
net = SSD(phase='train', # 訓練モード
          cfg=ssd_cfg)   # 設定値のdictオブジェクト

# SSDの初期の重みを設定
# VGG16の学習済み重みを取得
vgg_weights = torch.load(
    '/content/drive/MyDrive/Colab Notebooks/VOC2021_SSD/weights/vgg16_reducedfc.pth')
# SSDのvggネットワークの重みとしてロード
net.vgg.load_state_dict(vgg_weights)

# SSDモデルのvgg以外のネットワークの重みはHeの初期値で初期化
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)

# Heの初期値を適用
net.extras.apply(weights_init) # extrasネットワーク
net.loc.apply(weights_init)    # locネットワーク
net.conf.apply(weights_init)   #confネットワーク

# SSDモデルのサマリを出力
summary(
    net,
    # バッチを含む入力のサイズを設定
    input_size=(batch_size, 3, 300, 300),
    # レイヤーへの入力、出力のサイズとパラメーター数を出力
    col_names=['input_size','output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
SSD                                      --                        --                        --
├─ModuleList: 1-1                        --                        --                        --
├─ModuleList: 1-2                        --                        --                        --
├─ModuleList: 1-3                        --                        --                        --
├─ModuleList: 1-4                        --                        --                        --
├─ModuleList: 1-1                        --                        --                        --
│    └─Conv2d: 2-1                       [32, 3, 300, 300]         [32, 64, 300, 300]        1,792
│    └─ReLU: 2-2                         [32, 64, 300, 300]        [32, 64, 300, 300]        --
│    └─Conv2d: 2-3                       [32, 64, 300, 300]        [32, 64, 300, 300]        36,928
│    └─ReLU: 2-4            

In [ ]:
'''
6. 損失関数とオプティマイザーの作成
'''
import torch.optim as optim
from ssd import MultiBoxLoss

# 損失関数のオブジェクトを生成
criterion = MultiBoxLoss(
    jaccard_thresh=0.5, # 背景のDBoxに分類するときのIoUの閾値
    neg_pos=3, # 背景のDBoxの数はPositive DBoxの何倍にするか
    device=device) # ネットワークのTensorに割り当てるデバイス

# 勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = optim.SGD(
    net.parameters(),  # SSDモデルのパラメーター
    lr=1e-3,           # 学習率
    momentum=0.9,      # 慣性項に割り当てる係数
    weight_decay=5e-4) # 重み更新時のL2正則化の係数

In [ ]:
'''
7. SSDモデルの学習(パラメーターの更新)を行う関数
'''
import time
import pandas as pd

def train(net, dataloaders_dict, criterion, optimizer, num_epochs):
    '''
    Parameters:
      net(object): SSDモデル
      dataloaders_dict(objectのdict): データーローダー
      criterion(object): 損失関数
      optimizer(object): オプティマイザー
      num_epochs(object): 学習回数
    '''
    # SSDモデルでGPUを使用
    net.to(device)

    # ネットワークの構成に対して最適なアルゴリズムを見つけて高速化させる
    torch.backends.cudnn.benchmark = True

    iteration = 1          # イテレーション(ステップ)カウンター
    epoch_train_loss = 0.0 # 訓練1エポックごとの損失和
    epoch_val_loss = 0.0   # 検証1エポックごとの損失和
    logs = []              # 損失のログを記録するリスト

    # 学習、または検証のエポックごとのループ
    for epoch in range(num_epochs):
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        # 現在のエポック数を出力
        print('---------------------------------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('---------------------------------------')

        # エポック10回につき検証を1回行う
        for phase in ['train', 'val']:
            # エポックが10回に達するまではモデルを訓練モードにする
            if phase == 'train':
                net.train()  # モデルを訓練モードにする
            else:
                # エポックが10回に達していたらモデルを検証モードにして検証開始
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードにする
                    print('---------------------------------------')
                    print('（validation）')
                else:
                    # 10回に達していなければ次のエポックに進む
                    continue

            # 1ステップにおけるミニバッチを使用した学習または検証
            # データローダーをイテレートしてミニバッチを抽出
            for images, targets in dataloaders_dict[phase]:
                # 画像データにデバイスを割り当てる
                images = images.to(device)
                # 教師データ(正解BBoxのアノテーション情報)
                # (バッチサイズ, 物体数, 5[xmin, ymin, xmax, ymax, label_index])
                # にデバイスを割り当てる
                targets = [ann.to(device) for ann in targets]

                # optimizerが保持する勾配を0で初期化(累積しないように)
                optimizer.zero_grad()

                # 順伝搬（forward）とバックプロパゲーション(訓練時のみ)
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）を行って(loc, conf, dbox_list)を取得
                    # ・locの出力(バッチサイズ, 8732, 4[Δcx, Δcy, Δw, Δh])
                    # ・confの出力(バッチサイズ, 8732, 21)
                    # ・DBoxの情報(8732, 4[cx, cy, width, height])
                    outputs = net(images)

                    # Positive DBoxのオフセット情報の損失平均
                    # ミニバッチにおけるPositive DBoxの確信度の損失平均
                    loss_l, loss_c = criterion(outputs, targets)
                    # 2つの損失を合計する
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーションによるパラメーター更新を行う
                    if phase == 'train':
                        loss.backward()  # バックプロパゲーション

                        # 勾配が大きすぎると不安定になるので
                        # clipで勾配の上限を2.0に制限する
                        nn.utils.clip_grad_value_(net.parameters(),
                                                  clip_value=2.0)
                        # 勾配降下法の更新式を適用してバイアス、重みを更新
                        optimizer.step()

                        # ミニバッチを10個処理(10ステップ)ごとに損失を出力
                        if (iteration % 10 == 0):
                            # 10ステップの所要時間を取得
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            # ステップ数、損失、所要時間を出力
                            print('ステップ( {} )  loss: {:.4f} -- time: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        # エポックの損失をepoch_train_lossに加算する
                        epoch_train_loss += loss.item()
                        # ステップ数を1増やす
                        iteration += 1

                    # 検証モードでは順伝播後の損失の記録のみを行う
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        # エポック終了時の時刻を取得
        t_epoch_finish = time.time()
        print('---------------------------------------')
        # 訓練データの損失と検証データの損失を出力
        print('train_loss: {:.4f} - val_loss(Every 10 epochs): {:.4f}'.format(
            epoch_train_loss, epoch_val_loss))
        # エポック終了までに要した時間を取得
        print('time:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        # 次のエポックの開始時刻を取得
        t_epoch_start = time.time()

        # エポックごとに損失をdictオブジェクトに保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss,
                     'val_loss': epoch_val_loss}
        # ログのリストに追加
        logs.append(log_epoch)
        # ログのリストをデータフレームに変換
        df = pd.DataFrame(logs)
        # ログファイルに保存
        df.to_csv('/content/drive/MyDrive/Colab Notebooks/VOC2021_SSD/epoch_loss.csv')

        # 訓練時の損失和を0で初期化
        epoch_train_loss = 0.0
        # 検証時の損失和を0で初期化
        epoch_val_loss = 0.0

        # 1エポック終了ごとにモデルのパラメーター値を保存
        if ((epoch+1) % 10 == 0):
            torch.save(
                net.state_dict(),
                '/content/drive/MyDrive/Colab Notebooks/VOC2021_SSD/weights/ssd_weights' +
                str(epoch+1) + '.pth')
            print('--saved weights--')

In [ ]:
%%time
'''
8. VOC2012データセットをSSDモデルで学習
'''
num_epochs= 50 # 学習回数(エポック数)は50

train(net,                   # SSDモデル
      dataloaders_dict,      # データローダー
      criterion,             # 損失関数
      optimizer,             # オプティマイザー
      num_epochs=num_epochs) # エポック数

---------------------------------------
Epoch 1/50
---------------------------------------
ステップ( 10 )  loss: 16.3511 -- time: 19.2979 sec.
ステップ( 20 )  loss: 13.2701 -- time: 14.0174 sec.
ステップ( 30 )  loss: 10.0815 -- time: 14.2885 sec.
ステップ( 40 )  loss: 10.2929 -- time: 14.2889 sec.
ステップ( 50 )  loss: 11.2859 -- time: 14.3024 sec.
ステップ( 60 )  loss: 9.4437 -- time: 14.3668 sec.
ステップ( 70 )  loss: 8.8823 -- time: 13.9212 sec.
ステップ( 80 )  loss: 8.2995 -- time: 14.3094 sec.
ステップ( 90 )  loss: 8.7937 -- time: 14.1796 sec.
ステップ( 100 )  loss: 8.4553 -- time: 14.3252 sec.
ステップ( 110 )  loss: 8.0809 -- time: 14.1647 sec.
ステップ( 120 )  loss: 8.1018 -- time: 14.3356 sec.
ステップ( 130 )  loss: 7.7229 -- time: 14.5227 sec.
ステップ( 140 )  loss: 7.6691 -- time: 14.6427 sec.
ステップ( 150 )  loss: 9.8486 -- time: 14.8758 sec.
ステップ( 160 )  loss: 8.4884 -- time: 14.7506 sec.
ステップ( 170 )  loss: 10.0935 -- time: 14.5467 sec.
---------------------------------------
train_loss: 1791.4991 - val_loss(Every 10 epochs): 0.000